# Value Iteration

## Cas spécial du MPI

L'algorithme de Value Iteration calcule directement $v_*$, sans utiliser de policy. Il reprend en réalité la même structure que le Policy Iteration, mais ne met à jour qu'une seule fois $V(s), \forall s$ pour chaque évaluation. En effet il s'agit d'un cas spécial du Modified Policy Iteration, où $k=1$. Le pseudocode est alors le suivant : 

___

repeat, while $V$ stops updating:

&emsp;<strong>Improvement</strong><br/>
&emsp;for each $s$:<br/>
&emsp;&emsp;&emsp; $\pi(s) \leftarrow argmax_a [\sum_{s'} p(s'|s,a) [R_{ss'}^a + γ V(s')]]$

&emsp;<strong>Evaluation</strong><br/>
&emsp; for each $s$:<br/>
&emsp;&emsp;&emsp; $V(s) \leftarrow \sum_{a} \pi(a|s) \sum_{s'} p(s'|s,a) [R_{ss'}^a + γ V(s')]$

___

<em><span style="color:gray">L'improvement et l'évaluation ont ici été échangées de place, cela n'a aucune conséquence sur le résultat du programme car ces deux parties sont exécutées en boucle.</span></em>

## L'algorithme

Dénotons $a_*$ l'action optimale dans le state $s$. </br>
$\pi(s) = argmax_a [\sum_{s'} p(s'|s,a) [R_{ss'}^a + \gamma V(s')]] = argmax_a[Q(s,a)] = a_*$, donc </br>
$V(s) = max_a [\sum_{s'} p(s'|s,a) [R_{ss'}^a + \gamma V(s')]] = max_a [Q(s,a)] = \sum_{s'} p(s'|s,a_*) [R_{ss'}^{a_*} + \gamma V(s')]$

Ainsi, on peut réécrire le pseudocode comme suit:

___

repeat, while $V$ stops updating:

&emsp; for each $s$:<br/>
&emsp;&emsp; $V(s) \leftarrow max_a [\sum_{s'} p(s'|s,a) [R_{ss'}^a + \gamma V(s')]]$
</br>
___

Il est intéressant de remarquer que la règle de mise à jour de $V(s)$ est en réalité une application itérative de la Bellman optimality equation, ce qui fait qu'il calcule directement $v_*$. De plus, l'algorithme n'improve plus explicitement la policy $\pi$ car le calcul de $V(s)$ ne nécessite plus les probabilités d'actions choisies par une policy.

# Exemple Gridworld

In [1]:
import numpy as np

import envs.gridworld_dennybritz as grd

In [2]:
env = grd.GridworldEnv()

### Helper functions

#### Cette fonction sert d'interface avec l'environnement.
* `compute_q_value_for_s_a(env, V, s, a, gamma)` retourne, pour le state $s$ et l'action $a$ spécifiés la valeur $Q(s,a)$ soit $\sum_{s'} p(s'|s,a) [R_{ss'}^a + \gamma V(s')]$. <br>Cette fonction interroge la fonction `P[s][a]` de l'environnement qui renvoit une liste de tuple de la forme : `(p(s'|s,a), s', r(s,a,s'), done?)`. L'algorithme loop sur ces tuples et ajoute à `q` (équivalent à $Q(s,a)$) la valeur $p(s'|s,a) [R_{ss'}^a + \gamma V(s')]$.

In [3]:
def compute_q_value_for_s_a(env, V, s, a, gamma):
    q = 0
    
    for p_sPrime, sPrime, r_ss_a, done in env.P[s][a]:
        q += p_sPrime * (r_ss_a + gamma * V[sPrime])
        
    return q

### Initialization

La value function $V$ est initializée à des valeurs random.

Les hyperparamètres $\gamma$ and $\theta$ sont également initializés. Le discount factor prend la valeur de 1, cela ne posera pas de problème car l'horizon est dans ce cas là fini. Le threshold $\theta$ détermine l'écart maximal entre deux mises à jour de la value function à partir duquel l'algorithme aura considéré qu'il a convergé.

In [4]:
V = np.zeros([env.nS, 1])

gamma = 1.0
theta = 0.00001

### Iterations

Pour chaque itération, pour chaque state $s$, l'algorithme calcule les values de toutes les actions, $Q(s,a), \forall a$ :

\begin{align}
\large Q(s,a) \large \leftarrow \sum_{s'} p(s'|s,a) [R_{ss'}^a + \gamma V(s')]
\end{align}

Ce calcul est réalisé par la fonction `compute_q_value_for_s_a`. Toutes les valeurs $Q(s,a)$ sont stockées dans le tableau `q_s`, de dimensions $|A|$.

Une fois $Q(s,a)$ calculé pour toutes les actions, $V(s)$ prend la valeur maximale de ce tableau.

In [5]:
k=0
while True:
    k+=1
    delta = 0
    
    for s in range(env.nS):
        q_s = np.zeros([env.nA, 1])
        
        for a in range(env.nA):
            q_s[a] = compute_q_value_for_s_a(env, V, s, a, gamma)

        newV = np.max(q_s)
        delta = max(delta, np.abs(newV - V[s]))
        V[s] = newV
        
    if(delta < theta):
        print("Finished after " + str(k) + " iterations.")
        break

Finished after 4 iterations.


In [6]:
V

array([[ 0.],
       [-1.],
       [-2.],
       [-3.],
       [-1.],
       [-2.],
       [-3.],
       [-2.],
       [-2.],
       [-3.],
       [-2.],
       [-1.],
       [-3.],
       [-2.],
       [-1.],
       [ 0.]])

La value function $v_*$ a été trouvée :

<img src="img/gridworld_states_values_best.png" width="250" height="250" align="center"/>

La policy $\pi_*$ peut ainsi être facilement déduite de $v_*$, grâce à l'algorithme de policy improvement par exemple.